In [13]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [19]:
# Takes in Pressure, Crack length and Thickness
def calculateK (press,a,t):
    mapdl.clear()
    mapdl.prep7()
    
    # Define Material Property
    E = 210e9
    nu = 0.3
    YS = 240e6
    # Defining Geometry 
    # Corners of the rectangle
    m0 = mapdl.k("", 0, -0.1)
    m1 = mapdl.k("", 0,0.1002)
    m2 = mapdl.k("",0.0005,0.1002)
    m3 = mapdl.k("",0.0005,0.1)
    m4 = mapdl.k("", 0.1, 0.1)
    m5 = mapdl.k("", 0.1, -0.1)
    main = mapdl.a(m0,m1,m2,m3,m4,m5)
    # Crack geometry
    k0 = mapdl.k("",0,-0.00001)
    k1 = mapdl.k("",0.1*a,-0.00001)
    k2 = mapdl.k("",a,0)
    k3 = mapdl.k("",0.1*a,0.00001)
    k4 = mapdl.k("",0,0.00001)
    crack = mapdl.a(k0, k1, k2,k3,k4)
    # Substract the areas and extrude with Thickness t
    aout = mapdl.asba(main, crack)
    mapdl.vext(aout, dz=t)
    # mapdl.vplot(show_lines=True, quality=5)
    mapdl.et(1,"SOLID285")
    # Define the Mesh
    mapdl.smrtsize(4)
    mapdl.aesize("ALL", 0.003)
    mapdl.vmesh(1)
    scon1 = mapdl.k(10000,0.1,0,0)
    scon2 = mapdl.k(10001,0.2,0,0)
    
    # mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")
    
    # Using the SI units
    mapdl.units("SI")  # SI - International system (m, kg, s, K).

    # Define a material (nominal steel in SI)
    mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
    mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
    mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio

    # Fix the lower side.
    mapdl.nsel("S", "LOC", "Y", -0.1)
    mapdl.d("ALL", "UY")
    mapdl.d("ALL", "UZ")
    mapdl.d("ALL", "UX")

    mapdl.nsel("S", "LOC", "Y", 0.1)
    mapdl.cp(5, "UX", "ALL")

    mapdl.asel("S",'LOC',"Y",0.1002)
    mapdl.sfa('ALL', '', 'PRES', -press)
    mapdl.allsel(mute=True)
    mapdl.run("/SOLU")
    mapdl.antype("STATIC")
    mapdl.solve()
    mapdl.finish(mute=True)
    mapdl.post1()
    mapdl.path('P',3,5)
    node1 = mapdl.queries.node(a,0,0)
    node2 = mapdl.queries.node(0.5*a,0.0001,0)
    node3 = mapdl.queries.node(0,0.0001,0)
    mapdl.ppath(1, node1)
    mapdl.ppath(2, node2)
    mapdl.ppath(3, node3)
    mapdl.local(11,0,a,0,0)
    mapdl.csys(11)
    a = mapdl.kcalc(0,1,0,0)
    result = mapdl.result
    #result.plot_principal_nodal_stress(
    #    0,
    #    "SEQV",
    #    lighting=False,
    #    background="w",
    #    show_edges=True,
    #    text_color="k",
    #    add_text=False,
    #    )
    words = a.split(' ')
    
    # Extracting the K1 value from the String output
    for i in range(len(words)):
        if words[i]=='KI':
            x=i
    K1 = words[x+3][:-1] 
    print(f"K1 is equal to {K1}")
    return K1

In [20]:
P = [50e6,100e6,150e6,200e6]
#A = np.arange(0.01,0.015,0.001) 
A = [0.009,0.010, 0.011,0.012]
#T = np.arange(0.004,0.012,0.001)
T = [0.004, 0.005, 0.006,0.007]
data = np.zeros((len(P)*len(A)*len(T),4))
count =0
for i in range(len(P)):
    for j in range(len(A)):
        for k in range(len(T)):
            print(f"{P[i]}; {A[j]}; {T[k]}")
            K1 = calculateK(P[i],A[j],T[k])
            
            data[count,0] = P[i]
            data[count,1] = A[j]
            data[count,2] = T[k]
            data[count,3] = K1
            Area= 0.005*data[i,2]
            Force = mat[i,0]*Area
            mat[i,0] = Force
            count=count+1    
      
            
        

50000000.0; 0.009; 0.004


MapdlRuntimeError: 

Error in instance GRPC_127.0.0.1:50053

*** ERROR ***                           CP =       9.484   TIME= 19:11:45
 Volume 1 cannot be meshed.  26 location(s) found where non-adjacent     
 boundary triangles touch.  Geometry configuration may not be valid or   
 smaller element size definition may be required.                        

 *** ERROR ***                           CP =       9.484   TIME= 19:11:45
 Volume mesh failure - perhaps due to:                                   
  (1) Poorly shaped triangle facets: Mesh surfaces with triangular       
 elements, modify mesh control to get good triangular mesh, then VMESH   
 again.                                                                  
  (2) Complex geometry: Subdivide volume and try again.                  

 *** NOTE 

In [11]:
data[10:20,:]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [5]:
len(data)

1309

In [8]:
a =len(P)*len(A)*len(T)
a

1309

In [60]:
np.save('data54.npy',data)